In [1]:
from erddapy import ERDDAP
from erddapy import utilities

In [2]:
import json
import urllib
import requests
file = urllib.request.urlopen('https://raw.githubusercontent.com/IrishMarineInstitute/search-erddaps/master/erddaps.json')
servers = json.loads(file.read())
file.getcode()

200

In [3]:
servers=servers[:3]

In [4]:
print(servers)

[{'url': 'https://coastwatch.pfeg.noaa.gov/erddap/'}, {'url': 'https://apdrc.soest.hawaii.edu/erddap/'}, {'url': 'https://erddap.bco-dmo.org/erddap/'}]


In [5]:
time_min = '2010-07-10T00:00:00Z'
time_max = '2016-08-10T00:00:00Z'
bbox = [-72.0, -69, 38, 41]

kw = {
   'search_for': 'all',
   'min_lon': bbox[0],
   'max_lon': bbox[1],
   'min_lat': bbox[2],
   'max_lat': bbox[3],
   'min_time': time_min,
   'max_time': time_max,
}

In [14]:
import pandas as pd
all_datasets=pd.DataFrame()

for key in servers:
    url = key['url']
    url = url.rstrip("/")
    r = requests.get(url)
    try:
        r.raise_for_status()
        e = ERDDAP(
                 server=url,
                 protocol='tabledap',
                 response='csv'
           )
        print(e.get_search_url(**kw))
    except requests.exceptions.RequestException as err:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.HTTPError as errh:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.ConnectionError as errc:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.Timeout as errt:
        print("Bad ERDDAP!!! {}".format(url))
    datasets = pd.read_csv('%s'%e.get_search_url(**kw))
    all_datasets = pd.concat([all_datasets,datasets])

https://coastwatch.pfeg.noaa.gov/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1278720000.0&maxTime=1470787200.0&searchFor=all
https://apdrc.soest.hawaii.edu/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1278720000.0&maxTime=1470787200.0&searchFor=all
https://erddap.bco-dmo.org/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1278720000.0&maxTime=1470787200.0&searchFor=all


/anaconda3/envs/erddap/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [17]:
all_datasets

,Accessible,Background Info,Dataset ID,Email,FGDC,ISO 19115,Info,Institution,Make A Graph,RSS,Subset,Summary,Title,files,griddap,tabledap,wms
0,public,https://podaac.jpl.nasa.gov/dataset/AMSRE_L3_S...,jplAmsreSstMon_LonPM180,https://coastwatch.pfeg.noaa.gov/erddap/subscr...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/info/j...,Remote Sensing Systems,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,http://coastwatch.pfeg.noaa.gov/erddap/rss/jpl...,NaN,This data set contains sea surface temperature...,"AMSRE Model Output, obs4MIPs NASA-JPL, Global,...",NaN,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,NaN,https://coastwatch.pfeg.noaa.gov/erddap/wms/jp...
1,public,https://aquarius.nasa.gov/,jplAquariusSSS3MonthV5,https://coastwatch.pfeg.noaa.gov/erddap/subscr...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/info/j...,NASA/GSFC OBPG,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,http://coastwatch.pfeg.noaa.gov/erddap/rss/jpl...,NaN,This dataset has 3-month composites of Aquariu...,"Aquarius Sea Surface Salinity, L3 SMI, Version...",NaN,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,NaN,https://coastwatch.pfeg.noaa.gov/erddap/wms/jp...
2,public,https://aquarius.nasa.gov/,jplAquariusSSS7DayV5,https://coastwatch.pfeg.noaa.gov/erddap/subscr...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/info/j...,NASA/GSFC OBPG,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,http://coastwatch.pfeg.noaa.gov/erddap/rss/jpl...,NaN,This dataset has 7-day composites of Aquarius ...,"Aquarius Sea Surface Salinity, L3 SMI, Version...",NaN,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,NaN,https://coastwatch.pfeg.noaa.gov/erddap/wms/jp...
3,public,https://aquarius.nasa.gov/,jplAquariusSSSDailyV5,https://coastwatch.pfeg.noaa.gov/erddap/subscr...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/info/j...,NASA/GSFC OBPG,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,http://coastwatch.pfeg.noaa.gov/erddap/rss/jpl...,NaN,This dataset has daily composites of Aquarius ...,"Aquarius Sea Surface Salinity, L3 SMI, Version...",NaN,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,NaN,https://coastwatch.pfeg.noaa.gov/erddap/wms/jp...
4,public,https://aquarius.nasa.gov/,jplAquariusSSSMonthlyV5,https://coastwatch.pfeg.noaa.gov/erddap/subscr...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/metada...,https://coastwatch.pfeg.noaa.gov/erddap/info/j...,NASA/GSFC OBPG,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,http://coastwatch.pfeg.noaa.gov/erddap/rss/jpl...,NaN,This dataset has monthly composites of Aquariu...,"Aquarius Sea Surface Salinity, L3 SMI, Version...",NaN,https://coastwatch.pfeg.noaa.gov/erddap/gridda...,NaN,https://coastwatch.pfeg.noaa.gov/erddap/wms/jp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,public,http://apdrc.soest.hawaii.edu/thredds/dodsC/la...,hawaii_soest_2ee0_794e_6417,https://apdrc.soest.hawaii.edu/erddap/subscrip...,https://apdrc.soest.hawaii.edu/erddap/metadata...,https://apdrc.soest.hawaii.edu/erddap/metadata...,https://apdrc.soest.hawaii.edu/erddap/info/haw...,APDRC,https://apdrc.soest.hawaii.edu/erddap/griddap/...,http://apdrc.soest.hawaii.edu/erddap/rss/hawai...,NaN,satellite product Tropical Rainfall Measuring ...,satellite product TRMM TRMM PR 3G68 hourly,NaN,https://apdrc.soest.hawaii.edu/erddap/griddap/...,NaN,https://apdrc.soest.hawaii.edu/erddap/wms/hawa...
153,public,http://apdrc.soest.hawaii.edu/thredds/dodsC/la...,hawaii_soest_bbc1_f175_95f2,https://apdrc.soest.hawaii.edu/erddap/subscrip...,https://apdrc.soest.hawaii.edu/erddap/metadata...,https://apdrc.soest.hawaii.

In [8]:
# from erddapy import ERDDAP


# e = ERDDAP(
#   server='https://data.ioos.us/gliders/erddap',
#   protocol='tabledap',
# )

# e.response = 'csv'
# e.dataset_id = 'whoi_406-20160902T1700'
# e.constraints = {
#     'time>=': '2016-07-10T00:00:00Z',
#     'time<=': '2017-02-10T00:00:00Z',
#     'latitude>=': 38.0,
#     'latitude<=': 41.0,
#     'longitude>=': -72.0,
#     'longitude<=': -69.0,
# }
# e.variables = [
#     'depth',
#     'latitude',
#     'longitude',
#     'salinity',
#     'temperature',
#     'time',
# ]

In [9]:
    if url != 'http://erddap.emodnet-physics.eu/erddap' and \
    url != 'https://erddap.marine.ie/erddap' and \
    url != 'http://oos.soest.hawaii.edu/erddap' and \
    url != 'http://erddap.secoora.org/erddap' and \
    url != 'https://ecowatch.ncddc.noaa.gov/erddap' and \
    url != 'http://dap.onc.uvic.ca/erddap':

SyntaxError: unexpected EOF while parsing (<ipython-input-9-a1f1d5c2bc5d>, line 6)